# Intro

This notebook is designed to generate alerts so that you can work on the development environment.

It works as it is (just run blocks from the setup section), then you can run any section.

It is also possible to run the entire notebook at once: To do so, in the menu, click ‘Run’ then ‘Run All Cells’.

In [1]:
import ast
import pandas as pd
from dotenv import load_dotenv
import os
from api import get_token, get_camera_token
from pyroclient import Client
import glob
from PIL import Image
import numpy as np
import io
import itertools
import requests
import random
import shutil
from utils import dl_seqs_from_url, read_pred_file, send_triangulated_alerts
import time

In [2]:
API_URL = "http://api:5050"
load_dotenv("../.env")
SUPERADMIN_LOGIN = os.environ.get("SUPERADMIN_LOGIN")
SUPERADMIN_PWD = os.environ.get("SUPERADMIN_PWD")

# Get access token
admin_access_token = get_token(API_URL, SUPERADMIN_LOGIN, SUPERADMIN_PWD)
admin_client = Client(admin_access_token, API_URL)

In [3]:
users = pd.read_csv("../data/csv/API_DATA_DEV - users.csv")
cameras = pd.read_csv("../data/csv/API_DATA_DEV - cameras.csv")
users

,id,organization_id,password,login,role,created_at
0,2,2,test,test77,agent,2024-02-23T08:18:45.447773
1,3,3,test,test07,agent,2024-02-23T08:18:45.447773


In [4]:
cameras

,id,organization_id,name,angle_of_view,elevation,lat,lon,is_trustable,created_at
0,2,2,videlles-01,54.2,110.0,48.478300,2.424200,True,2023-11-07T15:07:19.226673
1,3,2,videlles-02,54.2,110.0,48.478300,2.424200,True,2023-11-07T15:07:19.226673
2,4,2,croix-augas-01,54.2,110.0,48.426700,2.710900,True,2023-11-07T15:07:19.226673
3,5,2,croix-augas-02,54.2,110.0,48.426700,2.710900,True,2023-11-07T15:07:19.226673
4,6,2,moret-sur-loing-01,54.2,110.0,48.379200,2.820800,True,2023-11-07T15:07:19.226673
5,7,2,moret-sur-loing-02,54.2,110.0,48.379200,2.820800,True,2023-11-07T15:07:19.226673
6,8,2,nemours-01,54.2,110.0,48.260500,2.706400,True,2023-11-07T15:07:19.226673
7,9,2,nemours-02,54.2,110.0,48.260500,2.706400,True,2023-11-07T15:07:19.226673
8,10,3,brison-01,87.0,110.0,44.545100,4.216500,True,2023-11-07T15:07:19.226673
9,11,3,brison-02,87.0,110.0,44.545100,4.216500,True,2023-11-07T15:07:19.226673


#### Download alerts sequences

In [5]:
BASE_DIRECTORY = "../data"

SAMPLE_PATH = "alert_samples"
url = "https://github.com/pyronear/pyro-envdev/releases/download/v0.0.1/alert_samples.zip"

if not os.path.isdir(os.path.join(BASE_DIRECTORY, SAMPLE_PATH)):
    print("Images not found, dowloading ...")
    #os.makedirs(SAMPLE_PATH, exist_ok=True)
    dl_seqs_from_url(url, os.path.join(BASE_DIRECTORY, SAMPLE_PATH))
else:
    print(f"SAMPLE_PATH {SAMPLE_PATH} directory exists, assume alerts inside, no download")


SAMPLE_PATH alert_samples directory exists, assume alerts inside, no download


## Send some random alerts sequences from a list of cameras
Download a file containing relevant images and predictions, then send alerts to selected cameras (with a random azimuth and random imgs picked from the alerts folder downloaded below).

In [6]:
send_alert_from_cam_ids = [2, 4, 5, 14] # select cameras

In [7]:
SAMPLE_PATH = "selection-true-positives"
url = "https://github.com/pyronear/pyro-envdev/releases/download/v0.0.1/selection-true-positives.zip"

if not os.path.isdir(SAMPLE_PATH):
    print("Images not found, dowloading ...")
    
    os.makedirs(SAMPLE_PATH, exist_ok=True)
    dl_seqs_from_url(url, SAMPLE_PATH)
else:
    print(f"SAMPLE_PATH {SAMPLE_PATH} directory exists, assume alerts inside, no download")

SAMPLE_PATH selection-true-positives directory exists, assume alerts inside, no download


In [8]:
def get_or_create_pose_id_for_azimuth(camera_client, camera_id, azimuth, tol=0.1, patrol_id=None):
    resp = camera_client.get_current_poses()
    resp.raise_for_status()
    poses = resp.json()
    for pose in poses:
        if abs(pose["azimuth"] - azimuth) <= tol:
            return pose["id"]

    create_resp = camera_client.create_pose(camera_id=camera_id, azimuth=azimuth, patrol_id=patrol_id)
    create_resp.raise_for_status()
    return create_resp.json()["id"]


send_alert_from_cam_ids = [2, 4, 5, 14]  # select cameras

sequances_folders = glob.glob(f"{SAMPLE_PATH}/*")

for camera_id in send_alert_from_cam_ids:
    camera_token = get_camera_token(API_URL, camera_id, admin_access_token)
    camera_client = Client(camera_token, API_URL)

    sequances_folder = sequances_folders[random.randint(0, len(sequances_folders) - 1)]
    imgs = glob.glob(f"{sequances_folder}/images/*")
    imgs.sort()
    preds = glob.glob(f"{sequances_folder}/labels_predictions/*")
    preds.sort()

    cam_center_azimuth = random.randint(0, 359)
    pose_id = get_or_create_pose_id_for_azimuth(camera_client, camera_id, cam_center_azimuth)

    print(f"Sending alerts from camera {camera_id} at azimuth {cam_center_azimuth} (pose_id={pose_id})")
    for img_file, pred_file in zip(imgs, preds):
        stream = io.BytesIO()
        im = Image.open(img_file)
        im.save(stream, format="JPEG", quality=80)

        bboxes = read_pred_file(pred_file)
        response = camera_client.create_detection(stream.getvalue(), bboxes, pose_id=pose_id)
        response.json()["id"]


Sending alerts from camera 2 at azimuth 94 (pose_id=1)
Sending alerts from camera 4 at azimuth 61 (pose_id=2)
Sending alerts from camera 5 at azimuth 259 (pose_id=3)
Sending alerts from camera 14 at azimuth 104 (pose_id=4)


## Send custom alerts sequences

The  following code block helps you send some alerts/sequences from a specific diretory containing sequences data

- Create a directory with the name you want, ex ```alerts_with_style```
- In directory ```../data```or in current dir, there are some alerts folders downloaded
- Copy each inteterting alert dir (dir that contains a dir img, and a dir label predictions) and paste them to your ```alerts_with_style``` directory
- then below, fill the ```SEQUENCES_DIR_PATH``` with the path to you directory
- fill the CAM_MAPPING mapping dict : the first number in alert folder are dict keys that will map with camera id in database :  ex {2:78} -> all alert folder starting with 2 will create alerts for the camera of id 78
     - you can find cameras id in the object ```cameras``` (created at the beginning of this notebook), or look in the DB table cameras
     - If you put alert dir with prefix number that does not match, no alerts will be created
  

In [9]:
SEQUENCES_DIR_PATH = "../data/alert_samples/single_sequences/*"
sequences_directories = glob.glob(SEQUENCES_DIR_PATH)
CAM_MAPPING = {22: 13, 42: 7, 59: 5, 14: 11, 43: 8, 79: 14, 13: 10, 11: 15, 60: 2, 15: 12, 12: 9, 10: 16, 41: 2, 65: 8}

def get_or_create_pose_id_for_azimuth(camera_client, camera_id, azimuth, tol=0.1, patrol_id=None):
    resp = camera_client.get_current_poses()
    resp.raise_for_status()
    for pose in resp.json():
        if abs(pose["azimuth"] - azimuth) <= tol:
            return pose["id"]

    create_resp = camera_client.create_pose(camera_id=camera_id, azimuth=azimuth, patrol_id=patrol_id)
    create_resp.raise_for_status()
    return create_resp.json()["id"]

for seq_path in sequences_directories:
    seq_prefix_num = int(seq_path.split("/")[-1].split("_")[0])

    if CAM_MAPPING.get(seq_prefix_num) is None:
        print(
            f" === WARNING the prefix {seq_prefix_num} is not in CAM_MAPPING dict, "
            "hence no alerts will be generated (or from random camera but no implemented yet)"
        )
        continue

    camera_id = CAM_MAPPING[seq_prefix_num]
    camera_token = get_camera_token(API_URL, camera_id, admin_access_token)
    camera_client = Client(camera_token, API_URL)

    imgs = sorted(glob.glob(f"{seq_path}/images/*"))
    preds = sorted(glob.glob(f"{seq_path}/labels_predictions/*"))

    cam_center_azimuth = random.randint(0, 359)
    pose_id = get_or_create_pose_id_for_azimuth(camera_client, camera_id, cam_center_azimuth)

    print(f"Sending alerts from camera {camera_id} at azimuth {cam_center_azimuth} (pose_id={pose_id})")
    for img_file, pred_file in zip(imgs, preds):
        stream = io.BytesIO()
        im = Image.open(img_file)
        im.save(stream, format="JPEG", quality=80)

        with open(pred_file, "r", encoding="utf-8") as file:
            bboxes = ast.literal_eval(file.read())

        response = camera_client.create_detection(stream.getvalue(), bboxes, pose_id=pose_id)
        time.sleep(0.5)
        response.json()["id"]


Sending alerts from camera 7 at azimuth 233 (pose_id=5)
Sending alerts from camera 5 at azimuth 153 (pose_id=6)
Sending alerts from camera 13 at azimuth 328 (pose_id=7)
Sending alerts from camera 7 at azimuth 354 (pose_id=8)
Sending alerts from camera 7 at azimuth 278 (pose_id=9)
Sending alerts from camera 11 at azimuth 194 (pose_id=10)
Sending alerts from camera 8 at azimuth 309 (pose_id=11)
Sending alerts from camera 7 at azimuth 212 (pose_id=12)
Sending alerts from camera 7 at azimuth 323 (pose_id=13)
Sending alerts from camera 7 at azimuth 329 (pose_id=14)


## Send 2 sequences that triangulate (user test07)
Download a file containning 2 alerts from 2 cameras that triangulate, then it send alerts to the api. 

In [10]:
BASE_DIRECTORY = "../data"
SAMPLE_PATH = "triangulated_sequences"
url = "https://github.com/pyronear/pyro-envdev/releases/download/v0.0.1/triangulated_sequences.zip" 
if not os.path.isdir(os.path.join(BASE_DIRECTORY, SAMPLE_PATH)):
    print("Images not found, dowloading ...")
    dl_seqs_from_url(url, os.path.join(BASE_DIRECTORY, SAMPLE_PATH))
else:
    print(f"SAMPLE_PATH {SAMPLE_PATH} directory exists, assume alerts inside, no download")


cam_triangulation = {
    "12": {
        "azimuth": 226,
        "path": os.path.join(BASE_DIRECTORY, SAMPLE_PATH, "brison-03")
    },
    "13": {
        "azimuth": 54,
        "path": os.path.join(BASE_DIRECTORY, SAMPLE_PATH, "serre-de-barre-01")
    }
}

send_triangulated_alerts(cam_triangulation, API_URL, Client, admin_access_token)


SAMPLE_PATH triangulated_sequences directory exists, assume alerts inside, no download
Cam 12: 10 images, 10 preds
Cam 13: 8 images, 8 preds
Send some entrelaced detections


NameError: name 'ast' is not defined

## Send 2 other sequences that triangulates (user test07)

In [ ]:
cam_triangulation = {
    "11": {
        "azimuth": 226,
        "path": "../data/alert_samples/triangulated_sequences/14_brison-03_2025-09-29_13-58-36"
    },
    "13": {
        "azimuth": 173,
        "path": "../data/alert_samples/triangulated_sequences/22_serre-de-barre-01_2025-09-29_13-56-58"
    }
}


send_triangulated_alerts(cam_triangulation, API_URL, Client, admin_access_token)

## Send 3 sequences that triangulates (user test77)

In [ ]:
cam_triangulation = {
    "4": {
        "azimuth": 190,
        "path": "../data/alert_samples/triangulated_sequences/41_croix-augas-02_2025-07-14_11-24-31"
    },
    "8": {
        "azimuth": 25,
        "path": "../data/alert_samples/triangulated_sequences/65_nemours-02_2025-07-14_11-24-10"
    },
    "5": {
        "azimuth": 280,
        "path": "../data/alert_samples/triangulated_sequences/66_moret-sur-loing-01_2025-07-14_11-22-05"
    }
}

send_triangulated_alerts(cam_triangulation, API_URL, Client, admin_access_token)
